## Project Setup

In [2]:
# all import statements needed for the project
import os 
import requests
import re 
import time 
import urllib.request
from bs4 import BeautifulSoup
from joblib import Parallel, delayed
import math 
import numpy as np 
import pandas as pd 
import geopandas as gpd
import pyarrow.parquet as pq

## Part 1: Data Preprocessing

In [ ]:
import os 
import requests
import re 
import time 
import urllib.request
from bs4 import BeautifulSoup

In [ ]:
def url_date_ge(a: str, b: str) -> bool:
  """Return a boolean that tells whether the input date a is greater than or equal to date b"""
  return (int(a[:4]), int(a[-2:])) >= (int(b[:4]), int(b[-2:]))

def url_date_le(a: str, b: str) -> bool:
  """Return a boolean that tells whether the input date a is less than or equal to date b"""
  return (int(a[:4]), int(a[-2:])) <= (int(b[:4]), int(b[-2:]))  

In [ ]:
# List out url that will be using
uber_url = 'https://drive.google.com/file/d/1F7D82w1D5151GXCR6BTEk7mNQ_YnPNDk/view?usp=sharing'
weather_url = 'https://drive.google.com/drive/folders/1I_Cj3RFHRGcQjb5Gas06buqRbKodIwKC?usp=sharing'
taxi_url = 'https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page'

In [ ]:
def download_yellow_taxi_parquet(taxi_url: str, 
                                 from_date: str = '2009-01', to_date: str = '2015-06',
                                 taxi_par_save_dir: str = 'yellow_taxi',
                                 max_waiting_time: float = 10 * 60)  -> None:
  """Download the yellow taxi parquet files from taxi_url."""
  # Make a request to the webpage
  response = requests.get(taxi_url)
  # Use BeautifulSoup to parse the HTML content of the webpage
  soup = BeautifulSoup(response.content, 'html.parser')
  data_urls = [tag['href'] for tag in soup.find_all(attrs={'title': 'Yellow Taxi Trip Records'})]
  inrange_urls = [_url for _url in data_urls 
                if url_date_ge(re.search(r'(\d+-\d+)', _url).group(0), from_date) 
                and url_date_le(re.search(r'(\d+-\d+)', _url).group(0), to_date)
                ]
  
  # Create folder
  if not os.path.exists(taxi_par_save_dir):
    os.mkdir(taxi_par_save_dir)

  done = False
  st = time.time()
  while not done and time.time() - st < max_waiting_time:
    try:
      for _url in inrange_urls:
        save_file = os.path.join(taxi_par_save_dir, _url[_url.index('yellow'):])
        if not os.path.exists(save_file):
          print(f'downloading to {save_file}...')
          urllib.request.urlretrieve(_url, save_file)
          print('finished.')
        else:
          print(f'{save_file} already downloaded.')
      
      done = True
      print('All downloads finished.')
    except:
      continue 
  
  if not done:
    print('Not all files downloaded. Might be insufficient max waiting time. You may re-run this function.')

In [ ]:
# Execute function to download yellow taxi parquet files
download_yellow_taxi_parquet(taxi_url)

In [ ]:
def download_unzip_shapefile(taxi_url: str, shapefile_save_dir: str = 'assets') -> None:
  """Download and unzip the Taxi Zone Shapefile from taxi_url."""
  # Make a request to the webpage
  response = requests.get(taxi_url)
  # Use BeautifulSoup to parse the HTML content of the webpage
  soup = BeautifulSoup(response.content, 'html.parser')
  shapefile_url = soup.find(string='Taxi Zone Shapefile').parent['href']

  if not os.path.exists(shapefile_save_dir):
    os.mkdir(shapefile_save_dir)
  save_file = os.path.join(shapefile_save_dir, 'taxi_zones.zip')
  if not os.path.exists(save_file):
    urllib.request.urlretrieve(shapefile_url, save_file)
  if not os.path.exists(os.path.join('.', 'taxi_zones.shp')):
    !unzip {save_file} 
  print('Downloaded shapefile.')

In [ ]:
# Execute the function to download and unzip the Taxi Zone Shapefile from taxi_url
download_unzip_shapefile(taxi_url)

In [ ]:
import os 
from joblib import Parallel, delayed
import math 
import numpy as np 
import pandas as pd 
import geopandas as gpd
import pyarrow.parquet as pq

class Part1:
  def __init__(self, 
               taxi_par_save_dir='yellow_taxi',
               uber_path='uber_rides_sample.csv',
               shapefile_save_dir='.',
               processed_path='processed'):
    self.taxi_par_save_dir = taxi_par_save_dir
    self.shapefile_save_dir = shapefile_save_dir
    self.shp_path = os.path.join(shapefile_save_dir, 'taxi_zones.shp')
    self.uber_path = uber_path
    self.processed_path = processed_path
    gdf = gpd.read_file(self.shp_path)
    gdf['centroid'] = gdf.geometry.centroid.to_crs(4326)
    gdf = gdf.set_index('OBJECTID')
    self.gdf = gdf
    self.pars = os.listdir(taxi_par_save_dir)
    self.pars = sorted(self.pars, key=lambda x: (int(x[x.index('.')-7: x.index('.')-3]), 
                                            int(x[x.index('.')-2: x.index('.')])))
    self.box = (min_longitude, max_longitude, min_latitude, max_latitude) = (-74.242330, -73.717047, 
                                                                             40.560445, 40.908524)
    
    # cols
    self.cols_mapping = {'vendor_name': 'vendor_id',
                        'VendorID': 'vendor_id',
                        'tpep_pickup_datetime': 'pickup_datetime',
                        'Trip_Pickup_DateTime': 'pickup_datetime',
                        'tpep_dropoff_datetime': 'dropoff_datetime',
                        'Trip_Dropoff_DateTime': 'dropoff_datetime',
                        'Passenger_Count': 'passenger_count',
                        'Trip_Distance': 'trip_distance',
                        'Start_Lon': 'pickup_longitude',
                        'Start_Lat': 'pickup_latitude',
                        'End_Lon': 'dropoff_longitude', 
                        'End_Lat': 'dropoff_latitude',
                        'Payment_Type': 'payment_type',
                        'Tip_Amt': 'tip_amount',
                        'Tolls_Amt': 'tolls_amount',
                        'Total_Amt': 'total_amount',
                        'Fare_Amt': 'fare_amount'
                        }
    self.cols = ['vendor_id', 'pickup_datetime', 'dropoff_datetime', 'passenger_count',
            'trip_distance', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
            'dropoff_latitude', 'payment_type', 'tip_amount', 'tolls_amount', 
            'total_amount', 'fare_amount']

    self.types = {'vendor_id': str, 'pickup_datetime': "datetime64[ns]", 'dropoff_datetime': "datetime64[ns]", 
            'passenger_count': np.int8, 'trip_distance': np.float32, 'pickup_longitude': np.float32,
            'pickup_latitude': np.float32, 'dropoff_longitude': np.float32, 'dropoff_latitude': np.float32,
            'payment_type': str, 'tip_amount': np.float32, 'tolls_amount': np.float32, 
            'total_amount': np.float32, 'fare_amount': np.float32}        
  
  def process_yellow_taxi_parquets(self)-> pd.DataFrame:
    """Reture a dataframe of all processed yellow taxi parquet files"""
    if not os.path.exists(self.processed_path):
      os.mkdir(self.processed_path)
    clean_sampled_df_list = [0] * len(self.pars)
    sample_ratio = self._determine_sample_ratio()
    for i, par in enumerate(self.pars):
      clean_sampled_df = self._process_one_yellow_taxi_parquet(par)
      clean_sampled_df_list[i] = clean_sampled_df
    
    processed_all_yellow_taxi_df = pd.concat(clean_sampled_df_list)
    return processed_all_yellow_taxi_df
  
  def _determine_sample_ratio(self) -> int:
    """Return a ratio, used for sampling yellow taxi data"""
    uber_df = pd.read_csv(self.uber_path)
    uber_rows = len(uber_df)
    num_rows = 0
    for i in range(len(self.pars)):
      data_path = os.path.join(self.taxi_par_save_dir, self.pars[i])
      pf = pq.ParquetFile(data_path)
      # count the number of rows in the file
      num_rows += pf.metadata.num_rows
    self.sample_ratio = uber_rows / num_rows
    return self.sample_ratio
  
  def _process_one_yellow_taxi_parquet(self, parquet_file: str) -> pd.DataFrame:
    """Return a processed dataset of the input parquet file
    
    Parameter:
    parquet_file: one single yellow taxi parquet file
    """
    data_path = os.path.join(self.taxi_par_save_dir, parquet_file)
    large_parquet = pq.ParquetFile(data_path)
    
    prod_list = []
    for trips in large_parquet.iter_batches(batch_size=100000):
      trips = trips.to_pandas()
      # Rename the columns by predefined column name in cols_mapping
      prod_trips = trips.rename(columns={k: v for k, v in self.cols_mapping.items() if k in trips.columns})
      # Lookup for latitude and longitude and add to dataframe
      if 'DOLocationID' in trips.columns:
        prod_trips = self.lookup_longlat('D', self.gdf, prod_trips)
      if 'PULocationID' in trips.columns:
        prod_trips = self.lookup_longlat('P', self.gdf, prod_trips)
      
      prod_trips = prod_trips[self.cols].astype(self.types).copy() # only a subset of columns are needed
      prod_trips = prod_trips.dropna(axis=0, how='any', subset=['dropoff_longitude', 'dropoff_latitude', 'pickup_longitude', 'pickup_latitude'])
      to_drop1 = prod_trips.apply(lambda row: not self._if_inside_box(row['dropoff_longitude'], row['dropoff_latitude']), axis=1)
      to_drop2 = prod_trips.apply(lambda row: not self._if_inside_box(row['pickup_longitude'], row['pickup_latitude']), axis=1)
      prod_trips = prod_trips.drop(prod_trips[to_drop1 + to_drop2].index)
      prod_trips['trip_distance'] = prod_trips.apply(lambda row: 
                                                    self.calc_distance(row['pickup_longitude'], row['pickup_latitude'], row['dropoff_longitude'], row['dropoff_latitude']) 
                                                    if np.isnan(row['trip_distance']) 
                                                    else row['trip_distance'],
                                                    axis=1)
      prod_trips = prod_trips.sample(frac=self.sample_ratio)
      prod_list.append(prod_trips)
    prod_trips = pd.concat(prod_list)

    par_savepath = os.path.join(self.processed_path, parquet_file)
    if not os.path.exists(par_savepath):
      prod_trips.to_parquet(par_savepath)
    return prod_trips

  def _if_inside_box(self, long: float, lat: float) -> bool:
    """Return a boolean that determines whether the trips start and/or end inside of the
    latitude/longitude coordinate box: (40.560445, -74.242330) and (40.908524, -73.717047).

    Parameter:
    long: float, the longtitude of the trip
    lat: float, the latitude of the trip
    """

    min_longitude, max_longitude, min_latitude, max_latitude = self.box 
    return min_longitude <= long <= max_longitude and min_latitude <= lat <= max_latitude
  
  def lookup_longlat(self, loc_type: str, gdf: pd.DataFrame, prod_trips: pd.DataFrame) -> pd.DataFrame:
    """Return a processed dataframe for yellow taxi, 
    Add addtional column to record corresponding latitude and longitude, for location ID 

    Parameter:
    loc_type: a string value, to indicate whether the input dataframe is prickup or dropoff, can be 'D' or 'P'
    gdf: a dataframe of Taxi Zone Shapefile, one location ID corresponds to one set of latitude and longitude
    prod_trips: a dataframe of yellow taxi location ID
    """

    # Create column names according to input as pickup or dropoff
    col_name = 'DOLocationID' if loc_type == 'D' else 'PULocationID'
    add_name = 'dropoff' if loc_type == 'D' else 'pickup'
    # Lookup for latitude and longitude for the location ID
    found, lookup = self.lookup_with_id(prod_trips[col_name], gdf)
    long, lat = found['centroid'].x, found['centroid'].y
    # Create new empty columns
    prod_trips[f'{add_name}_longitude'] = np.nan
    prod_trips[f'{add_name}_latitude'] = np.nan
    # Fill in the latitude and longitude value
    prod_trips.loc[lookup.index, f'{add_name}_longitude'] = long.values
    prod_trips.loc[lookup.index, f'{add_name}_latitude'] = lat.values
    return prod_trips 
  
  def lookup_with_id(self, sr: list, df: pd.DataFrame) -> tuple:
    """Look up for latitude and longitude of a given pickup and dropoff location ID
    Return a tuple containing the set of latitude and longitude, 
    and the valid location ID from yellow taxi data that corresponds to it 

    Parameter:
    sr: a series of yellow taxi pickup and dropoff location ID
    df: a dataframe of Taxi Zone Shapefile, one location ID corresponds to one set of latitude and longitude   
    """
    # filter for valid location ID in sr
    lookup = sr[sr.isin(df.index)]
    return df.loc[lookup], lookup
  
  def deg2rad(self, deg: float) -> float:
    """Convert a degree to radius
    Parameter:
    deg: float
    """
    return deg * (math.pi/180)

  def calc_distance(self, plong: float, plat: float, dlong: float, dlat: float) -> float:
    """Return a calculated distance in km between two latitude-longitude points
    Parameter:
    plong: float, pickup longitude
    plat: float, pickup latitude
    dlong: float, dropoff longitude
    dlat: float, dropoff latitude
    """
    # The following code was adapted from this StackOverflow 
    # answer from user1921: https://stackoverflow.com/questions/27928/calculate-distance-between-two-latitude-longitude-points-haversine-formula
    R = 6371 # Radius of the earth in km
    long = plong - dlong
    lat = plat - dlat
    lat_deg = self.deg2rad(lat)
    long_deg = self.deg2rad(long)
    a = (math.sin(lat_deg/2) * math.sin(lat_deg/2) 
    + math.cos(self.deg2rad(plat)) * math.cos(self.deg2rad(dlat)) 
    * math.sin(long_deg/2) * math.sin(long_deg/2)
    )
      
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a)); 
    d = R * c # Distance in km
    return d

In [ ]:
# Create an instance of the defined class to process yellow taxi data
p1 = Part1()
processed_all_yellow_taxi_df = p1.process_yellow_taxi_parquets()
# Save data to csv file
processed_all_yellow_taxi_df.to_csv('processed_yellow_taxi.csv')

## Part 2: Storing Cleaned Data

In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

def create_yellow_taxi() -> None:
    """Create a SQLite database, create and populate a table for sampled dataset of yellow taxi trips"""
    yellow_taxi_col_type = {'vendor_id': str, 
                'passenger_count': np.int8, 'trip_distance': np.float32, 'pickup_longitude': np.float32,
                'pickup_latitude': np.float32, 'dropoff_longitude': np.float32, 'dropoff_latitude': np.float32,
                'payment_type': str, 'tip_amount': np.float32, 'tolls_amount': np.float32, 
                'total_amount': np.float32, 'fare_amount': np.float32} 

    # Read the CSV file into a pandas DataFrame
    csv_file = 'processed_yellow_taxi.csv'
    ydf = pd.read_csv(csv_file, dtype=yellow_taxi_col_type, parse_dates=['pickup_datetime', 'dropoff_datetime'])

    # Connect to the SQLite database
    engine = create_engine('sqlite:///part2.sqlite3')

    # Write the DataFrame to an SQLite table
    table_name = 'yellow_taxi'
    ydf.to_sql(table_name, engine, if_exists='replace', index=False)

    engine.dispose()

def create_uber() -> None:
    """Create a SQLite database, create and populate a table for sampled dataset of uber trips"""
    f32 = np.float32
    uber_col_type = {
        'fare_amount': f32, 'pickup_longitude': f32, 'pickup_latitude': f32, 'dropoff_longitude': f32, 'dropoff_latitude': f32,
        'passenger_count': int, 'distance': f32
    }            

    # Read the CSV file into a pandas DataFrame
    csv_file = 'processed_uber.csv'
    udf = pd.read_csv(csv_file, dtype=uber_col_type, parse_dates=['key', 'pickup_datetime'])

    # Connect to the SQLite database
    engine = create_engine('sqlite:///part2.sqlite3')

    # Write the DataFrame to an SQLite table
    table_name = 'uber'
    udf.to_sql(table_name, engine, if_exists='replace', index=False)

    engine.dispose()


In [ ]:
# Call create_yellow_taxi() to create table for yellow taxi
create_yellow_taxi()

In [ ]:
# Call create_uber() to create table for yellow taxi
create_uber()

In [ ]:
def create_weather_hourly() -> None:
  """Create a SQLite database, create and populate a table for sampled dataset of hourly weather information"""
  # Read weather data
  string = str
  f32 = np.float32
  weather_col_type = {'STATION': int, 'LATITUDE': f32, 'LONGITUDE': f32, 
                      'ELEVATION': f32, 'NAME': string, 'REPORT_TYPE': string, 'SOURCE': str, 
                      'HourlyAltimeterSetting': string, 'HourlyDewPointTemperature': string,
                      'HourlyDryBulbTemperature': string, 'HourlyPrecipitation': string,
                      'HourlyPresentWeatherType': string, 'HourlyPressureChange': string,
                      'HourlyPressureTendency': f32, 'HourlyRelativeHumidity': f32,
                      'HourlySkyConditions': string, 'HourlySeaLevelPressure': string,
                      'HourlyStationPressure': string, 'HourlyVisibility': string, 
                      'HourlyWetBulbTemperature': f32,
                      'HourlyWindDirection': string, 'HourlyWindGustSpeed': f32,
                      'HourlyWindSpeed': f32,
                      'DailyWeather': string, 'DailySustainedWindDirection': f32,
                      'DailySnowfall': string, 'DailySnowDepth': string, 'DailyPrecipitation': string,
                      'DailyPeakWindSpeed': f32, 'DailyPeakWindDirection': f32,
                      'DailyMinimumDryBulbTemperature': f32, 'DailyMaximumDryBulbTemperature': f32,
                      'DailyHeatingDegreeDays': f32, 'DailyDepartureFromNormalAverageTemperature': f32,
                      'DailyCoolingDegreeDays': f32, 'DailyAverageWindSpeed': f32,
                      'DailyAverageWetBulbTemperature': f32, 'DailyAverageStationPressure': f32,
                      'DailyAverageSeaLevelPressure': f32, 'DailyAverageRelativeHumidity': f32,
                      'DailyAverageDryBulbTemperature': f32, 'DailyAverageDewPointTemperature': f32
                      }

  # Read all the CSV file into one pandas DataFrame
  wdfs = []                     
  for y in [2009, 2010, 2011, 2012, 2013, 2014, 2015]:
    wdf = pd.read_csv(f'{y}_weather.csv', parse_dates=['DATE'], dtype=weather_col_type, low_memory=False)
    wdfs.append(wdf)

  wdf_all = pd.concat(wdfs)

  # List out column names for hourly data
  hourly = [
      'STATION',
      'DATE',
      'LATITUDE',
      'LONGITUDE',
      'ELEVATION',
      'NAME',
      'REPORT_TYPE',
      'SOURCE',
      'HourlyAltimeterSetting',
      'HourlyDewPointTemperature',
      'HourlyDryBulbTemperature',
      'HourlyPrecipitation',
      'HourlyPresentWeatherType',
      'HourlyPressureChange',
      'HourlyPressureTendency',
      'HourlyRelativeHumidity',
      'HourlySkyConditions',
      'HourlySeaLevelPressure',
      'HourlyStationPressure',
      'HourlyVisibility',
      'HourlyWetBulbTemperature',
      'HourlyWindDirection',
      'HourlyWindGustSpeed',
      'HourlyWindSpeed',
      ]

  # List out column names for daily data
  daily = [
      'STATION',
      'DATE',
      'LATITUDE',
      'LONGITUDE',
      'ELEVATION',
      'NAME',
      'REPORT_TYPE',
      'SOURCE',
      'DailyAverageDewPointTemperature',
      'DailyAverageDryBulbTemperature',
      'DailyAverageRelativeHumidity',
      'DailyAverageSeaLevelPressure',
      'DailyAverageStationPressure',
      'DailyAverageWetBulbTemperature',
      'DailyAverageWindSpeed',
      'DailyCoolingDegreeDays',
      'DailyDepartureFromNormalAverageTemperature',
      'DailyHeatingDegreeDays',
      'DailyMaximumDryBulbTemperature',
      'DailyMinimumDryBulbTemperature',
      'DailyPeakWindDirection',
      'DailyPeakWindSpeed',
      'DailyPrecipitation',
      'DailySnowDepth',
      'DailySnowfall',
      'DailySustainedWindDirection',
      'DailySustainedWindSpeed',
      'DailyWeather'
  ]

  # Connect to the SQLite database
  engine = create_engine('sqlite:///part2.sqlite3')

  # Write the DataFrame to an SQLite table
  wdf_all[hourly].to_sql('hourly_weather', engine, if_exists='replace', index=False)
  
  engine.dispose()

In [ ]:
# Call create_weather() to create table for hourly and daily weather information
create_weather_hourly()

In [ ]:
import sqlite3
def create_weather_daily() -> None:
    """Create a SQLite database, create and populate a table for sampled dataset of daily weather information
    Calculated by applying aggregation function to hourly sql dataset"""
    weather_daily_query = """
        SELECT STATION,
        DATE(DATE) AS day,
        LATITUDE,
        LONGITUDE,
        ELEVATION,
        NAME,REPORT_TYPE,
        SOURCE,
        AVG(HourlyDewPointTemperature) AS DailyAverageDewPointTemperature,
        AVG(HourlyDryBulbTemperature) AS DailyAverageDryBulbTemperature,
        AVG(HourlyRelativeHumidity) AS DailyAverageRelativeHumidity,
        AVG(HourlySeaLevelPressure) AS DailyAverageSeaLevelPressure,
        AVG(CAST(HourlyStationPressure AS FLOAT)) AS DailyAverageStationPressure,
        AVG(HourlyWetBulbTemperature) AS DailyAverageWetBulbTemperature,
        AVG(HourlyWindSpeed) AS DailyAverageWindSpeed,
        MAX(HourlyDewPointTemperature) AS DailyMaximumDryBulbTemperature,
        MIN(HourlyDewPointTemperature) AS DailyMinimumDryBulbTemperature,
        MAX(CAST(HourlyWindDirection AS FLOAT)) AS DailyPeakWindDirection,
        MAX(HourlyWindSpeed) AS DailyPeakWindSpeed,
        SUM(HourlyPrecipitation) AS DailyPrecipitation,
        AVG(HourlyWindDirection) AS DailySustainedWindDirection,
        AVG(HourlyWindSpeed) AS DailySustainedWindSpeed
       
        FROM hourly_weather
        GROUP BY day
        """
    connection = sqlite3.connect('part2.sqlite3')
    weather_daily_df = pd.read_sql_query(weather_daily_query, connection)
    engine = create_engine('sqlite:///part2.sqlite3')
    weather_daily_df.to_sql('daily_weather', engine, if_exists='replace', index=False)
    engine.dispose()


In [ ]:
create_weather_daily()

In [ ]:
# Create schema file
create_table = """
CREATE TABLE IF NOT EXISTS daily_wealther (
	"STATION" BIGINT, 
	"DATE" DATETIME, 
	"LATITUDE" FLOAT, 
	"LONGITUDE" FLOAT, 
	"ELEVATION" FLOAT, 
	"NAME" TEXT, 
	"REPORT_TYPE" TEXT, 
	"SOURCE" TEXT, 
	"DailyAverageDewPointTemperature" FLOAT, 
	"DailyAverageDryBulbTemperature" FLOAT, 
	"DailyAverageRelativeHumidity" FLOAT, 
	"DailyAverageSeaLevelPressure" FLOAT, 
	"DailyAverageStationPressure" FLOAT, 
	"DailyAverageWetBulbTemperature" FLOAT, 
	"DailyAverageWindSpeed" FLOAT, 
	"DailyCoolingDegreeDays" FLOAT, 
	"DailyDepartureFromNormalAverageTemperature" FLOAT, 
	"DailyHeatingDegreeDays" FLOAT, 
	"DailyMaximumDryBulbTemperature" FLOAT, 
	"DailyMinimumDryBulbTemperature" FLOAT, 
	"DailyPeakWindDirection" FLOAT, 
	"DailyPeakWindSpeed" FLOAT, 
	"DailyPrecipitation" TEXT, 
	"DailySnowDepth" TEXT, 
	"DailySnowfall" TEXT, 
	"DailySustainedWindDirection" FLOAT, 
	"DailySustainedWindSpeed" FLOAT, 
	"DailyWeather" TEXT
)

;



CREATE TABLE IF NOT EXISTS hourly_wealther (
	"STATION" BIGINT, 
	"DATE" DATETIME, 
	"LATITUDE" FLOAT, 
	"LONGITUDE" FLOAT, 
	"ELEVATION" FLOAT, 
	"NAME" TEXT, 
	"REPORT_TYPE" TEXT, 
	"SOURCE" TEXT, 
	"HourlyAltimeterSetting" TEXT, 
	"HourlyDewPointTemperature" TEXT, 
	"HourlyDryBulbTemperature" TEXT, 
	"HourlyPrecipitation" TEXT, 
	"HourlyPresentWeatherType" TEXT, 
	"HourlyPressureChange" TEXT, 
	"HourlyPressureTendency" FLOAT, 
	"HourlyRelativeHumidity" FLOAT, 
	"HourlySkyConditions" TEXT, 
	"HourlySeaLevelPressure" TEXT, 
	"HourlyStationPressure" TEXT, 
	"HourlyVisibility" TEXT, 
	"HourlyWetBulbTemperature" FLOAT, 
	"HourlyWindDirection" TEXT, 
	"HourlyWindGustSpeed" FLOAT, 
	"HourlyWindSpeed" FLOAT
)

;



CREATE TABLE IF NOT EXISTS uber (
	"Unnamed: 0.1" BIGINT, 
	"Unnamed: 0" BIGINT, 
	"key" DATETIME, 
	fare_amount FLOAT, 
	pickup_datetime TIMESTAMP, 
	pickup_longitude FLOAT, 
	pickup_latitude FLOAT, 
	dropoff_longitude FLOAT, 
	dropoff_latitude FLOAT, 
	passenger_count BIGINT, 
	distance FLOAT
)

;



CREATE TABLE IF NOT EXISTS yellow_taxi (
	"Unnamed: 0" BIGINT, 
	vendor_id TEXT, 
	pickup_datetime DATETIME, 
	dropoff_datetime DATETIME, 
	passenger_count SMALLINT, 
	trip_distance FLOAT, 
	pickup_longitude FLOAT, 
	pickup_latitude FLOAT, 
	dropoff_longitude FLOAT, 
	dropoff_latitude FLOAT, 
	payment_type TEXT, 
	tip_amount FLOAT, 
	tolls_amount FLOAT, 
	total_amount FLOAT, 
	fare_amount FLOAT, 
	distance FLOAT
)
;

"""

with open("schema.sql", "w") as schema_file:
    schema_file.write(create_table)